In [37]:
import requests
import sqlite3
import time
from datetime import datetime
import dateutil.parser
import json

In [ ]:
# do this to get the first token 

# goto:
# https://www.strava.com/oauth/authorize?client_id=121298&response_type=code&redirect_uri=http://localhost&approval_prompt=force&scope=activity:read_all



# get auth code from resulting url
# http://localhost/?state=&code=236616d076eecdbecd4147be329c46cefd1b2cd5&scope=read,activity:read_all


# client_id = '121298'
# client_secret = '003ac5e5560276a0a04412a5940d60aac9e0a6db'
# auth_code = 'cb206c4bb0da0afcc808e9a80721704a0f57e477'

# response = requests.post(
#     url="https://www.strava.com/oauth/token",
#     data={
#         'client_id': client_id,
#         'client_secret': client_secret,
#         'code': auth_code,
#         'grant_type': 'authorization_code'
#     }
# )

# tokens = response.json()

# access_token = tokens['access_token']
# refresh_token = tokens['refresh_token']



In [3]:
# do this to refresh the token

response = requests.post(
    'https://www.strava.com/api/v3/oauth/token',
    data={
        'client_id': client_id,
        'client_secret': client_secret,
        'grant_type': 'refresh_token',
        'refresh_token': refresh_token
    }
)

tokens = response.json()
access_token = tokens['access_token']
refresh_token = tokens['refresh_token']
headers = {
    'Authorization': f'Bearer {access_token}'
}

In [21]:
# check usage - still do not know when things reset
response = requests.get(
    'https://www.strava.com/api/v3/athlete',
    headers={'Authorization': f'Bearer {access_token}'}
)

print("Status:", response.status_code)
print("Rate Limit:", response.headers.get('X-RateLimit-Limit'))
print("Rate Usage:", response.headers.get('X-RateLimit-Usage'))

Status: 200
Rate Limit: 200,2000
Rate Usage: 8,12


In [46]:
# roughly 13 api calls right now
def check_for_new_activities(cache_location):
    conn = sqlite3.connect(cache_location)
    cursor = conn.cursor()
    cursor.execute("SELECT id FROM activities")
    cached_ids = set(row[0] for row in cursor.fetchall())

    cursor.execute("SELECT MAX(json_extract(data, '$.start_date')) FROM activities")
    latest_date = cursor.fetchone()[0]
    if latest_date:
        dt = dateutil.parser.isoparse(latest_date)
        after_timestamp = int(dt.timestamp())
    else:
        after_timestamp = 0  # If no cache, get all activities

    conn.close()

    print(f"{len(cached_ids)} activities already cached.")

    looking = True
    new_ids = []
    page = 1
    calls = 0
    while looking:
        params = {"per_page": 200,"page": page,"after": after_timestamp,}
        response = requests.get('https://www.strava.com/api/v3/athlete/activities', headers=headers, params=params)
        calls += 1
        if response.status_code != 200:
            print(f"API error: {response.status_code}")
            break
        time.sleep(1)
        activities = response.json()
        if not activities:
            looking = False
            break  # No more activities
        for act in activities:
            activity_id = act['id']
            if activity_id in cached_ids:
                continue  # Skip if already cached
            else:
                new_ids.append(activity_id)
        
        page += 1
            
    print(f"{len(new_ids)} new activities to add.")
    print(f'Burned {calls} API calls')
    return new_ids


In [47]:
def cache_new_activities(new_id_list,cache_location):
    conn = sqlite3.connect(cache_location)
    cursor = conn.cursor()
    print(f'{len(new_id_list)} API calls - ballpark {len(new_id_list)} seconds')
    for activity_id in new_id_list:
        detail_url = f"https://www.strava.com/api/v3/activities/{activity_id}"
        detail_response = requests.get(detail_url, headers=headers)

        if detail_response.status_code != 200:
            print(f"Failed to fetch details for activity {activity_id}")
            continue
        time.sleep(1) # be nice to the API
        data = detail_response.json()

        # --- 4. Save to DB ---
        cursor.execute("INSERT INTO activities (id, data) VALUES (?, ?)", (activity_id, json.dumps(data)))
        conn.commit()
        conn.close()

In [48]:
# main driver - takes about 60sec
new_ids = check_for_new_activities('strava_cache.db')
cache_new_activities(new_ids,'strava_cache.db')


2296 activities already cached.
0 new activities to add.
Burned 1 API calls
0 API calls - ballpark 0 seconds
